# Running Ollama with VDS Tunnel in Google Colab

## Настройка

1. В Colab добавьте секреты (значок ключа слева):
   - vds_host: IP вашего VDS
   - ssh_key: приватный SSH-ключ в base64 (конвертируйте командой `base64 -w0 tunnel_key`)

2. На VDS должны быть настроены:
   - Публичный ключ в `~/.ssh/authorized_keys`
   - Открытый порт 11434
   - Параметр `GatewayPorts yes` в `/etc/ssh/sshd_config`

3. Используйте GPU runtime (Runtime -> Change runtime type -> GPU)

## 1. Установка пакетов

In [ ]:
# Install Ollama
!curl https://ollama.ai/install.sh | sh

# Install CUDA and SSH
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers openssh-client

## 2. Настройка окружения и импорт зависимостей

In [ ]:
from google.colab import userdata
import os
import subprocess
import threading
import time
import base64

# Проверка наличия всех необходимых секретов
required_secrets = ['vds_host', 'ssh_key']
missing_secrets = [secret for secret in required_secrets if not userdata.get(secret)]
if missing_secrets:
    raise ValueError(f"Missing required secrets: {', '.join(missing_secrets)}\n"
                    "Add them in Colab: Left sidebar -> Key icon -> Add new secret\n"
                    "Required secrets:\n"
                    "- vds_host: IP адрес VDS\n"
                    "- ssh_key: SSH приватный ключ в base64")

# Создаем директорию .ssh и сохраняем ключ
ssh_dir = os.path.expanduser('~/.ssh')
os.makedirs(ssh_dir, mode=0o700, exist_ok=True)

# Декодируем и сохраняем SSH ключ
key_data = base64.b64decode(userdata.get('ssh_key')).decode()
key_path = os.path.join(ssh_dir, 'tunnel_key')
with open(key_path, 'w') as f:
    f.write(key_data)
os.chmod(key_path, 0o600)

# Конфигурация
VDS_HOST = userdata.get('vds_host')
VDS_USER = 'tunnel'  # Фиксированный пользователь из скрипта установки
LOCAL_PORT = 11434

# Set LD_LIBRARY_PATH for NVIDIA
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

## 3. Создание Modelfile

In [ ]:
# Создаем Modelfile
modelfile_content = """
FROM qwen:14b
PARAMETER num_ctx 128000
"""

with open('Modelfile', 'w') as f:
    f.write(modelfile_content)

# Создаем модель
!ollama create qwen-coder -f ./Modelfile

## 4. Вспомогательные функции

In [ ]:
def run_and_print_output(process):
    """Helper function to continuously read and print process output"""
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())

## 5. Запуск SSH туннеля и Ollama

In [ ]:
# Настройка Ollama для удаленного доступа
os.environ['OLLAMA_HOST'] = '0.0.0.0'
os.environ['OLLAMA_ORIGINS'] = '*'

# Создание SSH туннеля
ssh_command = f"ssh -N -R 0.0.0.0:{LOCAL_PORT}:localhost:{LOCAL_PORT} -o StrictHostKeyChecking=no -i {key_path} {VDS_USER}@{VDS_HOST}"
ssh_process = subprocess.Popen(
    ssh_command.split(),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Запуск Ollama
print('>>> starting ollama serve')
ollama_serve = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    env=dict(os.environ)
)

serve_thread = threading.Thread(target=run_and_print_output, args=(ollama_serve,))
serve_thread.daemon = True
serve_thread.start()

time.sleep(5)

## 6. Загрузка модели

In [ ]:
print('>>> starting ollama pull qwen2.5-coder:14b')
pull_process = subprocess.Popen(
    ['ollama', 'pull', 'qwen2.5-coder:14b'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

while True:
    output = pull_process.stdout.readline()
    if output == '' and pull_process.poll() is not None:
        break
    if output:
        print(output.strip())

## 7. Показать URL и поддерживать работу сервера

In [ ]:
print(f"\n=== Your Ollama server is available at ===")
print(f"http://{VDS_HOST}")
print("=====================================")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Shutting down...")
    ollama_serve.terminate()
    ssh_process.terminate()